### TODO
* Explain: Q-learning
* Explain: SARSA
* Explain: n-step methods
* Explain: cliff env.


### DONE
* code: Run and compare (on cliff)
* Explain: tabular


### NOTES
* Example 6.6: Cliff Walking


# The goal of reinforcement learning is to achieve goal-directed learning from interactions with an environment.
At each time step, $t$, the agent recieves a state observation $S_t$ and a reward $R_t$, and performns an action $A_t$, like so:

![Image of environment-agent interaction](img/env-agent.png)

The agent must learn to pick actions that maximize the total expected future reward, called the **return** $G_t$.
In practice we often use the discounted return instead of the actual return.
Discounted returns weights imminent rewards higher than rewards in the far future.
This is controlled by the discounting factor $\gamma \in [0, 1])$, like so:

$$
G_t = \sum_{k=0}^t \gamma^k R_{t+k+1}
$$



For simple cases, where the actions and observation space are small and discrete it is possible to use **tabular approaches**, where each possible state-action pair is enumerated (in a table).
Tabular approaches aren't applicable to real world problems, but they can be useful for inlluminating the fundamental principles of reinforcement learning.

This notebook describes tabular versions of two of the classical reinforcement learning algorithms: Q-learning and SARSA.

# The Algorithms
Most reinforcement learning mehtods involve estimating a **value function** — a function that estimate how good a given state or state-action pair is in terms of the expected return.
The return of a state naturally depends on the agents behavior in the future, which is called the agents **policy**, value functions are therefore defined with respect to a policy.
So the state-action pair, $(s,a)$, under behavior policy $\pi$ is denoted as $q_\pi(s,a)$.
Similarly the state value function is $v_\pi(s)$.
In the tabular case the value function is represented as a table, with one entry for each unique state-action pair.
The subsript is often omitted, as it is clear which policy is used.
Value functions are nice to work with, as they automatically take into account what will happen in the future, greedily optimizing the value function is therefore equivalent to maximizing the longterm expected reward.

Both Q-learning and SARSA estimate the state-action value function, and follow an **$\epsilon$-greedy** policy.
With probability $1-\epsilon$ the action with the highest value is selected - $\pi(s) = \max_a q(s,a)$, and with probability a random action is selected.
In this notebook we will use an $\epsilon$-greedy policy, with $\epsilon=0.1$ held constant, unless noted otherwise.
Using a stochastic policy is desirable as it ensures that we explore the entire state space.
$\epsilon$-greedy is one of the simplest (and quite naive) solutions to the **exploration-exploitation** dillemma, but it works well for small to medium problems (e.g. it is sufficent for many, but not all Atari games).

___

One of the most important ideas witihin reinforcement learning is that of **temporal difference** (TD) methods.
TD methods combine elements of dynamic programming and Monte Carlo methods (see Sutton and Barto [1] chapters 4 and 5), resulting in a methods that can solve environments with unknown dynamics in an online manner, i.e. by using existing estimates to perform the updates.

**TODO**: Describe the ideas behind TD

**TODO**: generalized policy iteration (GPI) - to solve the control problem

## SARSA

SARSA on policy TD-control method, meaning that the value function estimates the behavior policy.
The value-function estimates are updated using the following equation:
$$
q(S_t, A_t) = q(S_t, A_t) + \alpha \big[ 
R_{t+1} + \gamma q(S_{t+1}, A_{t+1}) - q(S_t, A_t)
\big]
$$

where $\alpha$ is the step-size parameter, 
$q(S_t, A_t)$ is the current estimate, which we are updating, and 
$R_{t+1} + \gamma q(S_{t+1}, A_{t+1}$ is the **target** of the update.
For terminal state $S_t+1$ the value function q(S_{t+1}, A_{t+1}) is defined as zero.

This algorithm is called **1-step tabular SARSA**, as the one true reward is used in the target, and the 

We are using an estimate () as the target of our update.

## Q-learning
Q-learrning update


## What is the difference?


**TODO**: TD Methods





# Environment: Cliff walker

**TODO**: Describe environment

In [ ]:
%matplotlib inline

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

import utils
from cliff import Cliff
from agents import TabularNStepSARSA, TabularNStepQLearning

# Experiments

Below we train run tabular Q-learning and then SARSA agents.

During training we monitor 
 * the highest action value for each possible state (left plot), i.e. value of the greedy action of the agent.
 * the movement as heatmap (right plot) i.e. the number of times the agent has visited each state.

In [ ]:
## Run settings
num_runs = 10  # Number of runs to average rewards over
eps_per_run = 500  # Number of episodes (terminations) per run
n = 1  # n parameter in n-step Bootstrapping

In [ ]:
TN_QLearning_rewards = []
env = Cliff()
for i in range(num_runs):
    TN_QLearning = TabularNStepQLearning(env.state_shape, env.num_actions, n=n)
    _, rewards = utils.run_loop(env, TN_QLearning, 'QLearning, n='+str(n), max_e=eps_per_run)
    TN_QLearning_rewards.append(rewards)

TN_QLearning_rewards = np.array(TN_QLearning_rewards)

In [ ]:
# Run the last QLearning agent using visualizations.
utils.run_loop(env, TN_QLearning, 'QLearning, n='+str(n), max_e=1, render=True)

In [ ]:
TN_SARSA_rewards = []
env = Cliff()
for i in range(num_runs):
    try:
        TN_SARSA = TabularNStepSARSA(env.state_shape, env.num_actions, n=n)
        _, rewards = utils.run_loop(env, TN_SARSA, 'SARSA, n='+str(n), max_e=eps_per_run)
        TN_SARSA_rewards.append(rewards)
    except KeyboardInterrupt:
        break

TN_SARSA_rewards = np.array(TN_SARSA_rewards)

In [ ]:
# Run the last SARSA agent using visualizations.
utils.run_loop(env, TN_SARSA, 'SARSA, n='+str(n), max_e=1, render=True)

For $n=1$ we see that the Q-learning agent learns the shortest path, right by the cliff, where as the SARSA agent learns 


# Discussion

The code cell below plots the (smoothed) average reward for Q-learning and SARSA as a function of episodes.
In the $n=1$ case we can clearly see that the risky 'shortes-path strategy' of the Q-learning agent doesn't payoff, and it generally recieves a lower reward.

In [ ]:
plt.figure()
include_sd = False # include standard deviation in plot
utils.reward_plotter(TN_QLearning_rewards, 'QLearning', 'r', include_sd=include_sd, smooth_factor=2)
utils.reward_plotter(TN_SARSA_rewards, 'SARSA', 'b', include_sd=include_sd, smooth_factor=2)

axes = plt.gca()
axes.set_ylim([-100, 0])

plt.show()

If we however change the epsilon to zero 

Q-learning is able to learn the underlying dynamics of the environemnt, and learns to approximate the value function for the optimal policy, not the policy it follows.

(Since both the environment and the agents are both deterministic we only have to run one episode)

In [ ]:
TN_QLearning.min_eps = 0
TN_SARSA.min_eps = 0

_, TN_QLearning_rewards_no_eps = utils.run_loop(env, TN_QLearning, 'QLearning, n='+str(n), max_e=1, update=False)
_, TN_SARSA_rewards_no_eps = utils.run_loop(env, TN_SARSA, 'SARSA, n='+str(n), max_e=1, update=False)
clear_output()

print("Q-learning rewards with epsilon = 0:", TN_QLearning_rewards_no_eps[0])
print("SARSA rewards with epsilon = 0:     ", TN_SARSA_rewards_no_eps[0])

# N-step bootstrapping

**TODO**: - n step boot strapping

**TODO**: Describe: Why does Q learn the bad way?

If we anneal the $\epsilon$ to zero over time both agents will eventually learn to follow the safe path.

Another way of helping the Q-learning agent is to increase the $n$ bootstrapping parameter, thus allowing it to account for longer consequences longer into the future.
For any $n>1$ the 


**TODO**: Describe: re-run everything, but with $n=5$

In [ ]:
## Run settings
n = 5
# We leave the other settings as before

In [ ]:
TN_QLearning_rewards = []
env = Cliff()
for i in range(num_runs):
    TN_QLearning = TabularNStepQLearning(env.state_shape, env.num_actions, n=n)
    _, rewards = utils.run_loop(env, TN_QLearning, 'QLearning, n='+str(n), max_e=eps_per_run)
    TN_QLearning_rewards.append(rewards)
TN_QLearning_rewards = np.array(TN_QLearning_rewards)



In [ ]:
TN_SARSA_rewards = []
env = Cliff()
for i in range(num_runs):
    try:
        TN_SARSA = TabularNStepSARSA(env.state_shape, env.num_actions, n=n)
        _, rewards = utils.run_loop(env, TN_SARSA, 'SARSA, n='+str(n), max_e=eps_per_run)
        TN_SARSA_rewards.append(rewards)
    except KeyboardInterrupt:
        break
TN_SARSA_rewards = np.array(TN_SARSA_rewards)

In [ ]:
plt.figure()
include_sd = False # include standard deviation in plot
utils.reward_plotter(TN_QLearning_rewards, 'QLearning', 'r', include_sd=include_sd, smooth_factor=2)
utils.reward_plotter(TN_SARSA_rewards, 'SARSA', 'b', include_sd=include_sd, smooth_factor=2)

axes = plt.gca()
axes.set_ylim([-100, 0])

plt.show()


# Bibliographic Notes
[1] Richard S. Sutton and Andrew G. Barto. 1998. Introduction to Reinforcement Learning (1st ed.). MIT Press, Cambridge, MA, USA.